<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clustering-based" data-toc-modified-id="Clustering-based-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clustering based</a></span><ul class="toc-item"><li><span><a href="#modeling" data-toc-modified-id="modeling-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>modeling</a></span></li><li><span><a href="#evaluation" data-toc-modified-id="evaluation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>evaluation</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np 
from sklearn import metrics
seeds = 2021
# np.random.seed(seeds)

In [2]:
import pandas as pd
train = pd.read_json('../data/structured_train.json')
test = pd.read_json('../data/structured_test.json')

In [3]:
# train = train.groupby('label').sample(50, random_state=seeds)
# test = test.groupby('label').sample(50, random_state=seeds)

In [4]:
select_cols = ["global_index", "doc_path", "label",
               "reply", "reference_one", "reference_two", "tag_reply", "tag_reference_one", "tag_reference_two",
               "Subject", "From", "Lines", "Organization", "contained_emails", "long_string", "text", "error_message"
               ]
print("\nmay use cols: \n", select_cols)
train = train[select_cols]
test = test[select_cols]


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


# Clustering based
- Steps:
    1. Transform into TF-IDF matrix
    2. Dimension reduction into 200
    3. Clustering in cosine similarity space (since it is word)
    4. Assign labels with majority vote based on training set labels
    5. Prediction
        1. Transform test set into TF-IDF matrix
        2. Dimension reduction into 200
        3. Make prediction based on the clusters and mapping between clusters and labels from training set
    6. Evaluation
        1. Based on classification report
        
- Time complexity 
    - O(n^(dk+1)) where n is the number of observatons, d is the dimensionality, and k is k clusters

## modeling

In [5]:
# train_text = train['tag_reply']
# train_text = train['tag_reply']+ ' ' + train['tag_reference_one']
train_text = train['reply'] + ' ' + train['reference_one']
# train_text = train['reply'] 
train_label = train['label']

# test_text  = test['tag_reply'] 
# test_text  = test['tag_reply'] + ' ' + test['tag_reference_one']
test_text  = test['reply'] + ' ' + test['reference_one']
# test_text  = test['reply'] 
test_label = test['label']


In [6]:
from clustering_utils import *
# tfidf_vectorizer, dimension_reduction

In [7]:
# dtm_train, dtm_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, min_df=2)
# dtm_train, transform_mapper = dimension_reduction(dtm_train, out_dim=2)
# dtm_test = transform_mapper.transform(dtm_test)

# print('dtm_train.shape', dtm_train.shape)
# print('dtm_test.shape', dtm_test.shape)
# # print(word_to_idx)
# clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=30, repeats=2)
# pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)
# print(metrics.classification_report(y_true = test_label, y_pred=pred))

## evaluation

In [14]:
dtm_train, dtm_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, min_df=2)
dtm_train, transform_mapper = dimension_reduction(dtm_train, out_dim=50)
dtm_test = transform_mapper.transform(dtm_test)

print('dtm_train.shape', dtm_train.shape)
print('dtm_test.shape', dtm_test.shape)
# print(word_to_idx)
# clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=25, repeats=2)
# clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=25, metric='L2', repeats=2)
pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)
print(metrics.classification_report(y_true = test_label, y_pred=pred))

num of words: 38680
Dimension reduction with truncate SVD:
   input columns with  38680
   output columns with  50
dtm_train.shape (11083, 50)
dtm_test.shape (7761, 50)
Group to label mapping: 
Group 0 <-> label comp.sys.mac.hardware
Group 1 <-> label sci.space
Group 2 <-> label comp.graphics
Group 3 <-> label sci.electronics
Group 4 <-> label comp.windows.x
Group 5 <-> label sci.crypt
Group 6 <-> label soc.religion.christian
Group 7 <-> label misc.forsale
Group 8 <-> label misc.forsale
Group 9 <-> label rec.motorcycles
Group 10 <-> label sci.med
Group 11 <-> label talk.politics.mideast
Group 12 <-> label comp.sys.ibm.pc.hardware
Group 13 <-> label talk.politics.mideast
Group 14 <-> label misc.forsale
Group 15 <-> label comp.windows.x
Group 16 <-> label rec.sport.baseball
Group 17 <-> label comp.sys.ibm.pc.hardware
Group 18 <-> label sci.space
Group 19 <-> label comp.sys.ibm.pc.hardware
Group 20 <-> label soc.religion.christian
Group 21 <-> label talk.politics.guns
Group 22 <-> label r

C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)